In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\a1831\Desktop\NBS_Project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.900369449497976, 'Fn': 1.184383282690447, 'kn': 0.702366454588397, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 0.1, 'l': 0.8, 'cn': 1.0415462755795253, 'Fn': 1.7637226511189565, 'kn': 0.6949789883617077, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 0.1, 'l': 0.8, 'cn': 0.932903940019097, 'Fn': 2.1321345406869066, 'kn': 0.26579121798441974, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 0.1, 'l': 0.8, 'cn': 0.8278081871246422, 'Fn': 3.8199302863134226, 'kn': 0.3800624712417908, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 0.1, 'l': 0.8, 'cn': 1.0432905838092195, 'Fn': 3.913825949541608, 'kn': 0.4753646534341449, 'omega_n': 1, 'Rn': 7.948101841125762}
{'s': 0.1, 'l': 0.8, 'cn': 0.9766658416311175, 'Fn': 2.4920855793255217, 'kn': 0.6194093753579976, 'omega_n': 1, 'Rn': 3.859406010253179}
{'s': 0.1, 'l': 0.8, 'cn': 0.833279172556006, 'Fn': 1.5048957788256199, 'kn': 0.3873734616989366, 'omega_n': 1, 'Rn': 7.130152926250668}
{'s': 0.1, 'l': 0.8, 'cn': 0.9124006604

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_uni[i]) for i,md in enumerate(MDs)]))

Uniform λ: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
Uniform p: [0.57361785 0.49202044 0.48782256 1.04131136 0.88412906 0.57245759
 0.6350686  0.63100999 0.76698469 0.82634447]
Uniform Dmax: 0.9258162626243813
Uniform r: [2.49720758 2.49720758 2.49720758 2.49720758 2.49720758 2.49720758
 2.49720758 2.49720758 2.49720758 2.49720758]
social welfare: 24.972075763699145
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.4849674937208897', 'MD 2 Dn: 0.8791292568302455', 'MD 3 Dn: 0.925816262624381', 'MD 4 Dn: 0.13557703443194546', 'MD 5 Dn: 0.17782681347955617', 'MD 6 Dn: 0.48979288119047076', 'MD 7 Dn: 0.35435113125604023', 'MD 8 Dn: 0.35628022997598957', 'MD 9 Dn: 0.2300656757143794', 'MD 10 Dn: 0.20511122341909174']
Dmax: 0.9258162626243813  D0: 1
max Dn: 0.925816262624381
Dmax == max Dn: False
Dn<=Dmax: True
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 24.972075763697095 Q_star: 32.59978295939449
rn>=Ln: True
sum Ln: 7.627707195695345


In [4]:
T = 100
N = len(MDs)
lam_pro, p_pro, r_pro, Dmax_pro = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.proportional_baseline(esp,MDs,43)
    lam_pro += l
    p_pro += p 
    r_pro += r
    Dmax_pro += d
lam_pro, p_pro, r_pro, Dmax_pro = lam_pro/T, p_pro/T, r_pro/T, Dmax_pro/T
print(f"Proportional λ: {lam_pro}")
print(f"Proportional p: {p_pro}")  
print(f"Proportional Dmax: {Dmax_pro}")
print(f"Proportional r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_pro[i]) for i,md in enumerate(MDs)]))

Proportional λ: [2.09025991 3.11270752 3.76289958 6.74160743 6.90731927 4.39815949
 2.65591668 5.8750097  6.85821515 7.59790528]
Proportional p: [0.37120698 0.34235891 0.38925098 1.15725469 1.01879669 0.52647414
 0.47573815 0.69731903 0.90315535 1.0154291 ]
Proportional Dmax: 0.9217225381449619
Proportional r: [1.17300071 1.74677231 2.11164356 3.78321867 3.87621194 2.46813528
 1.49043291 3.29690606 3.84865596 4.26375125]
social welfare: 28.058728658821067
sum lambda: 49.99999999999998  lambda0: 50
['MD 1 Dn: 0.4163687895676022', 'MD 2 Dn: 0.8668176229530887', 'MD 3 Dn: 0.9217225381449594', 'MD 4 Dn: 0.14029795567433845', 'MD 5 Dn: 0.18417768723312586', 'MD 6 Dn: 0.48404477432674053', 'MD 7 Dn: 0.31790147597589336', 'MD 8 Dn: 0.36190435608389293', 'MD 9 Dn: 0.23774407179012644', 'MD 10 Dn: 0.21374197834225211']
Dmax: 0.9217225381449619  D0: 1
max Dn: 0.9217225381449594
Dmax == max Dn: False
Dn<=Dmax: True
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 28.05872865882013 Q_star:

In [5]:
lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_non[i]) for i,md in enumerate(MDs)]))

c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


✅ 这个点严格满足所有约束（在容差范围内）。
p_lo > F[idx]: MD 0 : 20.798409790489178>1.184383282690447
p_lo > F[idx]: MD 1 : 4.732300319048444>1.7637226511189565
p_lo > F[idx]: MD 2 : 6.449346264011636>2.1321345406869066
p_lo > F[idx]: MD 3 : 5.0417685522498>3.8199302863134226
p_lo > F[idx]: MD 4 : 5.562652144654886>3.913825949541608
p_lo > F[idx]: MD 5 : 36.98946712558912>2.4920855793255217
p_lo > F[idx]: MD 6 : 6.055216335467328>1.5048957788256199
p_lo > F[idx]: MD 7 : 4.88040096692385>3.328898592922507
p_lo > F[idx]: MD 8 : 5.401903734476152>3.8860025666261127
p_lo > F[idx]: MD 9 : 7.948154725344748>4.305125863998515
p_lo > F[idx]: MD 0 : 20.798409790489178>1.184383282690447
p_lo > F[idx]: MD 1 : 4.732300319048444>1.7637226511189565
p_lo > F[idx]: MD 2 : 6.449346264011636>2.1321345406869066
p_lo > F[idx]: MD 3 : 5.0417685522498>3.8199302863134226
p_lo > F[idx]: MD 4 : 5.562652144654886>3.913825949541608
p_lo > F[idx]: MD 5 : 36.98946712558912>2.4920855793255217
p_lo > F[idx]: MD 6 : 6.055216335467328>1.

ValueError: The lower bound exceeds the upper bound.

In [6]:
lam_con, p_con, r_con, Dmax_con = baseline.contract_baseline(esp,MDs)
print(f"Contract λ: {lam_con}")
print(f"Contract p: {p_con}")  
print(f"Contract Dmax: {Dmax_con}")
print(f"Contract r: {r_con}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_con, p_con))
print("sum lambda:", np.sum(lam_con), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_con[i],lam_con[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_con, " D0:", esp.D0)
max_Dn = max(md.delay(p_con[i], lam_con[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_con == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_con[i], lam_con[i]) <= Dmax_con for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_con <= esp.D0)
print("pn<=Fn:", all(p_con[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_con[i] <= p_con[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_con), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_con[i] >= md.Ln(p_con[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_con[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -85.82915570707715
            Iterations: 48
            Function evaluations: 1553
            Gradient evaluations: 48
Contract λ: [3.02197806 3.14798901 6.70999662 8.10839358 5.44103078 4.42393646
 6.08672705 3.14750598 6.66900402 3.24343843]
Contract p: [0.48718925 0.48686685 0.77542479 0.88444414 0.67227671 0.60065194
 0.72495009 0.48697859 0.77015796 0.49995143]
Contract Dmax: 0.35014256145952494
Contract r: [0.56370769 0.54517557 0.69939336 0.80605967 0.6354476  0.62881447
 0.70126842 0.58791574 0.71059593 0.59847067]
social welfare: 85.82915570694092
sum lambda: 49.99999999999999  lambda0: 50
['MD 1 Dn: 0.35014256146006645', 'MD 2 Dn: 0.35014256146028105', 'MD 3 Dn: 0.3501425614596211', 'MD 4 Dn: 0.3501425614600894', 'MD 5 Dn: 0.35014256145989997', 'MD 6 Dn: 0.3501425614598697', 'MD 7 Dn: 0.35014256145970957', 'MD 8 Dn: 0.3501425614602996', 'MD 9 Dn: 0.3501425614596

c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [7]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_swm[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -84.1841915351389
            Iterations: 39
            Function evaluations: 874
            Gradient evaluations: 39
SWM λ: [3.64752469 3.71790396 6.16501379 6.99898543 5.31790865 4.62240538
 5.74966126 3.79466578 6.12610362 3.85982745]
SWM p: [0.53703154 0.53227532 0.73163574 0.79550558 0.6622391  0.61632587
 0.69779537 0.53856641 0.72653866 0.54906916]
SWM Dmax: 0.35041131362855193
Optimization terminated successfully    (Exit mode 0)
            Current function value: -24.99275310911575
            Iterations: 10
            Function evaluations: 110
            Gradient evaluations: 10
SWM r: [7.79972596 7.77348013 7.77928568 7.81584884 7.76955185 7.80572566
 7.80760681 7.82663849 7.7906958  7.83285263]
social welfare: 85.80794205976217
sum lambda: 49.99999999999999  lambda0: 50
['MD 1 Dn: 0.3504103136293584', 'MD 2 Dn: 0.3504103136285748', 'MD 3 Dn: 0.35041031363099

In [8]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_opt[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -103.07402380971322
            Iterations: 148
            Function evaluations: 4748
            Gradient evaluations: 147
Optimal λ: [4.99945818 5.00042229 4.99999963 4.99931794 4.99992457 4.99986143
 5.00042551 4.99986182 5.00005923 5.0006694 ]
Optimal p: [0.48225578 0.68052686 0.48982108 0.48225459 0.4810134  0.48211699
 0.7884177  0.83248046 1.14781972 0.96816907]
Optimal r: [729027.12962918 663168.12982059 648092.09980928 492845.44185562
 455267.47217088 582372.05931531 710928.49849925 487621.69176755
 471302.70121993 422450.14641778]
Optimal Dmax: 0.999999
social welfare: -4999944.302876328
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.9962492271709585', 'MD 2 Dn: 0.2949692302861799', 'MD 3 Dn: 0.9055477881314837', 'MD 4 Dn: 0.982778035961964', 'MD 5 Dn: 0.999998999997743', 'MD 6 Dn: 0.9999990000072636', 'MD 7 Dn: 0.22000683504419535', 'MD 8 Dn: 0.19494958225201386', 'M

In [9]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_admm[i]) for i,md in enumerate(MDs)]))

MD SLSQP failed: Positive directional derivative for linesearch
MD SLSQP failed: Positive directional derivative for linesearch
MD SLSQP failed: Positive directional derivative for linesearch
ADMM λ: [3.97892479 4.02812466 5.87446402 6.45718355 5.24605455 4.72122931
 5.56763421 4.11882808 5.8427473  4.16480953]
ADMM p: [0.56343481 0.5569929  0.70828734 0.75205955 0.65638817 0.62412057
 0.68312975 0.56439932 0.70376775 0.5733628 ]
ADMM Dmax: 0.35055579362047856
ADMM r: [7.84486287 7.81543201 7.74084264 7.74965078 7.75774326 7.81671034
 7.78202658 7.87033705 7.7530519  7.87365362]
social welfare: 85.78068095325504
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.35055503892797774', 'MD 2 Dn: 0.3505555331010742', 'MD 3 Dn: 0.35055731617995645', 'MD 4 Dn: 0.3505572276038099', 'MD 5 Dn: 0.3505567467423378', 'MD 6 Dn: 0.3505567633033208', 'MD 7 Dn: 0.35055670856922266', 'MD 8 Dn: 0.35055534566286667', 'MD 9 Dn: 0.35055690249890176', 'MD 10 Dn: 0.35055576876574074']
Dmax: 0.35055579362047856  D0: 1

In [10]:
print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("Nash Product Non-cooperative:",model.nash_product_log(esp, MDs, lam_non, p_non, r_non))
print("Nash Product Contract:",model.nash_product_log(esp, MDs, lam_con, p_con, r_con))
print("=============================")
print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("SW Non-cooperative:",model.social_welfare(esp, MDs, lam_non, p_non))
print("SW Contract:",model.social_welfare(esp, MDs, lam_con, p_con))
print("=============================")
print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)
print("Dmax Non-cooperative:", Dmax_non)
print("Dmax Contract:", Dmax_con)

Nash Product Opt: nan
Nash Product SWM: 24.992753367717455
Nash Product ADMM: 24.98894037277331
Nash Product Proportional: 10.414149841741715
Nash Product Uniform: 9.440202808067802
Nash Product Non-cooperative: nan
Nash Product Contract: nan
SW Opt: -4999944.302876328
SW SWM: 85.80794205976217
SW ADMM: 85.78068095325504
SW Proportional: 28.058728658821067
SW Uniform: 24.972075763699145
SW Non-cooperative: 67.3048780229474
SW Contract: 85.82915570694092
Dmax Opt: 0.999999
Dmax SWM: 0.35041131362855193
Dmax ADMM: 0.35055579362047856
Dmax Proportional: 0.9217225381449619
Dmax Uniform: 0.9258162626243813
Dmax Non-cooperative: 0.09178816014683396
Dmax Contract: 0.35014256145952494


c:\Users\a1831\Desktop\NBS_Project\src\model.py:11: RuntimeWarning: invalid value encountered in log
  n0 = ESP.omega_0 * np.log(ESP.Q(Dmax)-np.sum(r))
c:\Users\a1831\Desktop\NBS_Project\src\model.py:12: RuntimeWarning: invalid value encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
